**ÉTAPE 0** : préparation des données

In [1]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True)

There are no NaN values in the dataframe


**ÉTAPE 1** : Phase d'entraînement

1. Obtention de la matrice de corrélation des actifs sur une fenêtre arrière de 30 jours (1 mois)

In [5]:
number_of_repetitions = 40
lookback_window = 365
number_of_clusters = 20

consolidated_W = process.consolidated_W(lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=number_of_clusters, number_of_repetitions=number_of_repetitions)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project

KeyError: 'AA'

In [4]:

portfolio_return = process.portfolio_annualized_returns(evaluation_window=evaluation_window, df_cleaned=df_cleaned, training_window=L, final_weights=W)

UnboundLocalError: cannot access local variable 'portfolio_return' where it is not associated with a value

In [12]:
portfolio_return = (portfolio_return + 1)**(250/L) - 1

In [13]:
risk_free_rate = 0.04
SR, PNL = process.Sharpe_and_PnL(portfolio_daily_returns=portfolio_return, risk_free_rate=risk_free_rate)

In [14]:
import plotly.graph_objects as go


process.plot_PnL(SR=SR, PNL=PNL)

In [80]:

# Création du tableau synthétique
data = {
    'Rendement Moyen': [Rp],
    'Écart Type': [sigma_p],
    'Taux sans Risque': [Rf],
    'Sharpe Ratio': [SR],
    'PNL Finale': [PNL[-1]]
}

table = pd.DataFrame(data)

# Affichage du tableau
print(table)

   Rendement Moyen  Écart Type  Taux sans Risque  Sharpe Ratio  PNL Finale
0         0.016567    0.156385              0.02     -0.021955    0.496999
